In [12]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import time
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

import random, math
from nsga2 import Solution
from nsga2 import NSGAII

% matplotlib inline

In [23]:
# need ffmpeg
plt.rcParams['animation.ffmpeg_path'] = r'C:\ffmpeg-20180221-a246701-win64-static\bin\ffmpeg.exe'

Define test function

In [5]:
class ZDT1Solution(Solution):
    '''
    Solution for the ZDT1 function.
    '''
    def __init__(self):
        '''
        Constructor.
        '''
        Solution.__init__(self, 2)
        
        self.xmin = 0.0
        self.xmax = 1.0
        
        for _ in range(30):
            self.attributes.append(random.random())
        
        self.evaluate_solution()
        
    def evaluate_solution(self):
        '''
        Implementation of method evaluate_solution() for ZDT1 function.
        '''
        self.objectives[0] = self.attributes[0]
        
        sum = 0.0
        for i in range(1, 30):
            sum += self.attributes[i]
            
        g = 1.0 + (9.0 * (sum / 29))
        
        self.objectives[1] = g * (1.0 - math.sqrt(self.attributes[0] / g))
        
    def crossover(self, other):
        '''
        Crossover of ZDT1 solutions.
        '''
        child_solution = ZDT1Solution()
        
        for i in range(30):
            child_solution.attributes[i] = math.sqrt(self.attributes[i] * other.attributes[i])
        
        return child_solution
    
    def mutate(self):
        '''
        Mutation of ZDT1 solution.
        '''
        self.attributes[random.randint(0, 29)] = random.random()

In [9]:
nsga2 = NSGAII(2, 0.9, 0.03)

P = []

n_population = 100
n_iterations = 500

for i in range(n_population):
    P.append(ZDT1Solution())
    
Q = nsga2.initialise(P)

Ps = []
t0 = time.time()
for ii in range(n_iterations):
    if ii % 100 == 0:
        print('Iteration', ii, '- Elapsed time:', time.time() - t0)
    P, Q = nsga2.generate(P, Q, n_population, n_iterations)
    Ps.append(copy.deepcopy(P))

Iteration 0 - Elapsed time: 0.0
Iteration 100 - Elapsed time: 6.126793622970581
Iteration 200 - Elapsed time: 12.282664775848389
Iteration 300 - Elapsed time: 18.553843021392822
Iteration 400 - Elapsed time: 24.693170309066772


Visualise final generation 

In [27]:
fig, ax = plt.subplots(figsize = (10, 6))

f1 = np.linspace(0, 1, 1000)
f2 = 1.0 - np.sqrt(f1)
ax.plot(f1, f2, 
        alpha = 0.8,
        label = 'Pareto Optimal Curve')

ax.set_ylabel('Objective f2(x)')
ax.set_xlabel('Objective f1(x)')
ax.grid(which = 'both')
ax.legend()
ax.set_xlim([0, 1])

population, = ax.plot([p.objectives[0] for p in Ps[0]], [p.objectives[1] for p in Ps[0]], 
                      'x', 
                      color = 'r',
                      alpha = 0.8,
                      markersize = 2,
                      label = 'Population')
ax.set_xlim([0, 1])
ax.set_ylim([0, 5])
def animate(i):
    population.set_data([p.objectives[0] for p in Ps[i]], [p.objectives[1] for p in Ps[i]])
    
anim = FuncAnimation(fig, animate, interval = 100, frames = len(Ps))

# prevent plot from showing
plt.close()

Make a video showing the population pareto front converging to the ideal optimal curve

In [25]:
HTML(anim.to_html5_video())